In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus310.JPG
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus207.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus367.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus300.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus140.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus68.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus362.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus374.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus290.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus106.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus151.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus58.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus57.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus155.jpg
/kaggle/input/cotton-leaf-dataset/cotton/fussarium_wilt/fus386.jp

In [4]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from torchvision.models import vgg16, VGG16_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/kaggle/input/cotton-leaf-dataset/cotton" 

# data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomRotation(30),  
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into train, validation, and test 
train_size = int(0.5 * len(dataset))  
val_size = int(0.4 * len(dataset))  
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained VGG16
weights = VGG16_Weights.DEFAULT
model = vgg16(weights=weights)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Dropout(0.5),  
    nn.Linear(128, 4)  
)
model = model.to(device)

#loss func and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)  
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5) 

# Training 
def train_model(model, train_loader, val_loader, num_epochs=10):
    final_validation_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()  

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        final_validation_accuracy = accuracy  
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Validation Accuracy: {accuracy * 100:.2f}%")

    torch.save(model.state_dict(), "/kaggle/working/best_model_VG16.pth") 
    print("Training complete. Final Validation Accuracy: {:.2f}%".format(final_validation_accuracy * 100))
    print("Model weights saved to '/kaggle/working/best_model_VG16.pth'")

# Test
def test_model(model, test_loader):
    model.load_state_dict(torch.load("/kaggle/working/best_model_VG16.pth"))
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print("Test Accuracy: {:.2f}%".format(accuracy * 100))

train_model(model, train_loader, val_loader, num_epochs=10)
test_model(model, test_loader)

Epoch 1/10, Loss: 27.2329, Validation Accuracy: 87.57%
Epoch 2/10, Loss: 12.6427, Validation Accuracy: 91.67%
Epoch 3/10, Loss: 6.9558, Validation Accuracy: 93.86%
Epoch 4/10, Loss: 6.6218, Validation Accuracy: 94.01%
Epoch 5/10, Loss: 5.4875, Validation Accuracy: 96.35%
Epoch 6/10, Loss: 2.1252, Validation Accuracy: 97.51%
Epoch 7/10, Loss: 1.9105, Validation Accuracy: 98.39%
Epoch 8/10, Loss: 0.8550, Validation Accuracy: 97.37%
Epoch 9/10, Loss: 1.8125, Validation Accuracy: 97.22%
Epoch 10/10, Loss: 0.9197, Validation Accuracy: 95.03%
Training complete. Final Validation Accuracy: 95.03%
Model weights saved to '/kaggle/working/best_model_VG16.pth'


<ipython-input-4-7271aa6936aa>:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/best_model_VG16.pth"))


Test Accuracy: 97.08%
